In [2]:
import numpy as np
import pandas as pd
import itertools
from scipy.optimize import minimize, differential_evolution
from scipy.linalg import block_diag
from sklearn.model_selection import KFold

In [4]:
file_path = './Synched_Data_GR0_22_DEN_MAXZ1_25/NEWDATA/'
file_date = ['101922', '102122', '111422', '111622', '120522', '120722', 
                '013023', '020123', '031323', '031523', '041723', '041923', '061523']
date = file_date[0]

file_name = f'DAYUBIGR_{date}_GR0_22_DEN_032825_V2392628911.CSV'
full_path = file_path + file_name

raw_data = pd.read_csv(full_path, header=None, names=['SUBJECTID', 'TIME', 'X', 'Y', 'Z'])
clear_data = raw_data.reset_index(drop=True)
clear_data = clear_data[(clear_data["X"] <= 15) & (clear_data["Y"] <= 9) & 
                        (clear_data["X"] >= 0) & (clear_data["Y"] >= 0)].copy()
target_subject_base = "DS_STARFISH_2223_27"
subject_data = clear_data[clear_data['SUBJECTID'].str.startswith(target_subject_base)].copy()
subject_data['TIME'] = pd.to_datetime(subject_data['TIME'])
t0 = subject_data['TIME'].min()
subject_data['timestamp'] = (subject_data['TIME'] - t0).dt.total_seconds()

subject_data['side'] = subject_data['SUBJECTID'].str.extract(r'(\d+[LR])$')[0].str[-1].map({'L': 'left', 'R': 'right'})
subject_data['timestamp_rounded'] = subject_data['timestamp'].round(3)

grouped = subject_data.groupby('timestamp_rounded')
real_data = []

for ts, group in grouped:
    entry = {'timestamp': ts}
    left = group[group['side'] == 'left']
    right = group[group['side'] == 'right']
    
    if not left.empty:
        left_xy = left[['X', 'Y']].iloc[0].to_numpy()
        entry['left'] = left_xy
    if not right.empty:
        right_xy = right[['X', 'Y']].iloc[0].to_numpy()
        entry['right'] = right_xy
    
    if 'left' in entry and 'right' in entry:
        entry['observed'] = 'both'
        entry['obs'] = np.concatenate([entry['left'], entry['right']])
    elif 'left' in entry:
        entry['observed'] = 'left'
        entry['obs'] = entry['left']
    elif 'right' in entry:
        entry['observed'] = 'right'
        entry['obs'] = entry['right']
    else:
        entry['observed'] = 'none'
        entry['obs'] = np.array([])

    real_data.append(entry)

In [6]:
DT_VIRT = 0.5  # Virtual time step interval
SIGMA_MIN = 0.000001
SIGMA_MAX = 10
sigma_v_values = [0.01, 0.1, 0.5, 1]
sigma_omega_values = [0.001, 0.01, 0.1, 0.5]
sigma_obs_values = [0.5, 1]
param_combinations = list(itertools.product(sigma_v_values, sigma_omega_values, sigma_obs_values))
results = []

In [8]:
class RobustEKF:
    def __init__(self):
        self.SIGMA_MIN = 1e-6
        self.SIGMA_MAX = 10.0
        self.DT_VIRT = 0.5
        
    def multiple_initialization_optimization(self, data, timestamps, virtual_timestamps, n_initializations=10):
        best_result = None
        best_score = np.inf
        all_results = []
        
        # Define parameter bounds
        bounds = [
            (self.SIGMA_MIN, self.SIGMA_MAX),  # sigma_vx
            (self.SIGMA_MIN, self.SIGMA_MAX),  # sigma_vy
            (self.SIGMA_MIN, self.SIGMA_MAX),  # sigma_omega
            (0.01, self.SIGMA_MAX),            # sigma_obs
            (0.01, 1.0)                       # d
        ]
        
        # Generate diverse initial guesses
        initial_guesses = self._generate_initial_guesses(n_initializations, bounds)
        
        for i, init_params in enumerate(initial_guesses):
            print(f"Optimization attempt {i+1}/{n_initializations}")
            try:
                result_lbfgs = self._optimize_single(data, timestamps, virtual_timestamps, 
                                                   init_params, bounds, method='L-BFGS-B')
                result_nm = self._optimize_single(data, timestamps, virtual_timestamps, 
                                                init_params, bounds, method='Nelder-Mead')
                
                if result_lbfgs['score'] < result_nm['score']:
                    result = result_lbfgs
                    result['method'] = 'L-BFGS-B'
                else:
                    result = result_nm
                    result['method'] = 'Nelder-Mead'
                
                all_results.append(result)
                
                if result['score'] < best_score:
                    best_score = result['score']
                    best_result = result
                    
                print(f"  Score: {result['score']:.4f}, Method: {result['method']}")
                
            except Exception as e:
                print(f"  Failed: {str(e)}")
                continue
        
        # Try global optimization as fallback
        if best_result is None or len(all_results) < 3:
            print("Trying global optimization (Differential Evolution)...")
            try:
                global_result = self._global_optimization(data, timestamps, virtual_timestamps, bounds)
                if global_result['score'] < best_score:
                    best_result = global_result
                    best_result['method'] = 'Differential Evolution'
            except Exception as e:
                print(f"Global optimization failed: {str(e)}")
        
        return best_result, all_results
    
    def _generate_initial_guesses(self, n_initializations, bounds):
        initial_guesses = []
        
        # Common reasonable starting points
        reasonable_starts = [
            [0.1, 0.1, 0.01, 0.5, 0.23],    # Conservative
            [0.5, 0.5, 0.1, 1.0, 0.15],     # Moderate
            [1.0, 1.0, 0.5, 1.5, 0.3],      # Aggressive
            [0.01, 0.01, 0.001, 0.1, 0.1],  # Very conservative
        ]
        
        for start in reasonable_starts:
            if len(initial_guesses) < n_initializations:
                initial_guesses.append(start)
        
        # Random initializations for remaining slots
        np.random.seed(42)  # For reproducibility
        while len(initial_guesses) < n_initializations:
            random_params = []
            for low, high in bounds:
                # Use log-uniform distribution for better coverage
                if low > 0:
                    random_params.append(np.exp(np.random.uniform(np.log(low), np.log(high))))
                else:
                    random_params.append(np.random.uniform(low, high))
            initial_guesses.append(random_params)
        
        return initial_guesses
    
    def _optimize_single(self, data, timestamps, virtual_timestamps, initial_params, bounds, method):
        def objective_with_regularization(params):
            try:
                _, _, _, _, neg_log_likelihood = self.ekf_forward(data, timestamps, virtual_timestamps, params)
                
                # Add regularization to prevent extreme parameter values
                regularization = 0.001 * np.sum(np.log(params[:-1] + 1e-10))  # Exclude 'd' from regularization
                
                return neg_log_likelihood - regularization
            except Exception as e:
                print(f"Objective function error: {e}")
                return 1e10  # Large penalty for failed evaluations
        
        if method == 'Nelder-Mead':
            # Nelder-Mead doesn't support bounds directly, so we use penalties
            def penalized_objective(params):
                # Check bounds
                for i, (param, (low, high)) in enumerate(zip(params, bounds)):
                    if param < low or param > high:
                        return 1e10  # Large penalty for out-of-bounds
                return objective_with_regularization(params)
            
            result = minimize(penalized_objective, initial_params, method='Nelder-Mead',
                            options={'maxiter': 200, 'xatol': 1e-6, 'fatol': 1e-6})
        else:
            result = minimize(objective_with_regularization, initial_params, method=method,
                            bounds=bounds, options={'maxiter': 100})
        
        return {
            'params': result.x,
            'score': result.fun,
            'success': result.success,
            'nit': result.nit if hasattr(result, 'nit') else 0
        }
    
    def _global_optimization(self, data, timestamps, virtual_timestamps, bounds):
        """Global optimization using Differential Evolution"""
        def objective(params):
            try:
                _, _, _, _, neg_log_likelihood = self.ekf_forward(data, timestamps, virtual_timestamps, params)
                return neg_log_likelihood
            except:
                return 1e10
        
        result = differential_evolution(objective, bounds, seed=42, maxiter=50, 
                                      popsize=10, atol=1e-6, tol=1e-6)
        
        return {
            'params': result.x,
            'score': result.fun,
            'success': result.success,
            'nit': result.nit
        }
    
    def robust_cross_validation(self, data, timestamps, virtual_timestamps, initial_params, n_splits=3):
        """
        More robust cross-validation with better error handling
        """
        # Use smaller n_splits for stability
        data_array = np.array(data)
        timestamps_array = np.array(timestamps)
        
        # Ensure minimum data size per fold
        if len(data) < n_splits * 100:  # At least 100 points per fold
            n_splits = max(2, len(data) // 100)
            print(f"Reduced n_splits to {n_splits} due to limited data")
        
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
        fold_results = []
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(data_array)):
            print(f"Processing fold {fold + 1}/{n_splits}")
            
            try:
                train_data = data_array[train_idx].tolist()
                val_data = data_array[val_idx].tolist()
                train_timestamps = timestamps_array[train_idx].tolist()
                val_timestamps = timestamps_array[val_idx].tolist()
                
                # Create virtual timestamps for training
                min_time = min(train_timestamps)
                max_time = max(train_timestamps)
                train_virtual_timestamps = np.arange(min_time, max_time, self.DT_VIRT).tolist()
                
                # Optimize parameters on training fold
                best_result, _ = self.multiple_initialization_optimization(
                    train_data, train_timestamps, train_virtual_timestamps, n_initializations=5)
                
                if best_result is None:
                    raise ValueError("Optimization failed for this fold")
                
                # Evaluate on validation set
                val_virtual_timestamps = np.arange(min(val_timestamps), max(val_timestamps), self.DT_VIRT).tolist()
                _, _, _, _, val_score = self.ekf_forward(val_data, val_timestamps, 
                                                       val_virtual_timestamps, best_result['params'])
                
                fold_results.append({
                    'params': best_result['params'],
                    'val_score': val_score,
                    'train_score': best_result['score'],
                    'train_size': len(train_data),
                    'val_size': len(val_data),
                    'success': True
                })
                
                print(f"Fold {fold + 1} - Val score: {val_score:.2f}, Train score: {best_result['score']:.2f}")
                
            except Exception as e:
                print(f"Error in fold {fold + 1}: {str(e)}")
                fold_results.append({
                    'params': None,
                    'val_score': np.inf,
                    'train_score': np.inf,
                    'train_size': len(train_data) if 'train_data' in locals() else 0,
                    'val_size': len(val_data) if 'val_data' in locals() else 0,
                    'success': False
                })
        
        # Analyze results
        successful_folds = [fr for fr in fold_results if fr['success']]
        
        if not successful_folds:
            return {
                'avg_val_score': np.inf,
                'std_val_score': np.inf,
                'best_params': None,
                'all_fold_results': fold_results,
                'n_successful_folds': 0
            }
        
        val_scores = [fr['val_score'] for fr in successful_folds]
        avg_val_score = np.mean(val_scores)
        std_val_score = np.std(val_scores)
        
        # Select best parameters (lowest validation score)
        best_fold = min(successful_folds, key=lambda x: x['val_score'])
        best_params = best_fold['params']
        
        return {
            'avg_val_score': avg_val_score,
            'std_val_score': std_val_score,
            'best_params': best_params,
            'all_fold_results': fold_results,
            'n_successful_folds': len(successful_folds)
        }
    
    def validate_parameters(self, params):
        sigma_vx, sigma_vy, sigma_omega, sigma_obs, d = params
        
        issues = []
        
        # Check for extreme values
        if sigma_vx > 5.0 or sigma_vy > 5.0:
            issues.append("Velocity noise parameters are very high")
        
        if sigma_omega > 2.0:
            issues.append("Angular velocity noise is very high")
        
        if sigma_obs > 3.0:
            issues.append("Observation noise is very high")
        
        if d > 0.5:
            issues.append("Sensor separation distance seems too large")
        
        # Check for imbalanced parameters
        velocity_ratio = max(sigma_vx, sigma_vy) / min(sigma_vx, sigma_vy)
        if velocity_ratio > 10:
            issues.append("Velocity noise parameters are highly imbalanced")
        
        return issues
    
    def ekf_forward(self, data, timestamps, virtual_timestamps, params):
        """
        Your existing EKF forward function with improved numerical stability
        """
        sigma_vx, sigma_vy, sigma_omega, sigma_obs, d = params
        master_timestamps = sorted(set(timestamps + virtual_timestamps))
        T = len(master_timestamps)
        
        # Initialize arrays
        s_hat = [np.zeros(6)] * T
        P = [np.zeros((6, 6))] * T
        s_filt = [np.zeros(6)] * T
        P_filt = [np.zeros((6, 6))] * T
        neg_log_likelihood = 0.0
        
        # Improved initialization
        s_hat[0] = np.zeros(6)
        for entry in data[:min(10, len(data))]:
            if entry['observed'] != 'none':
                if entry['observed'] == 'both':
                    s_hat[0][:2] = (entry['left'] + entry['right']) / 2
                    break
                elif entry['observed'] in ['left', 'right']:
                    sensor_pos = entry['left'] if entry['observed'] == 'left' else entry['right']
                    s_hat[0][:2] = sensor_pos
                    break
        
        # More conservative initial covariance
        P[0] = np.diag([2.0, 2.0, np.pi, 2.0, 2.0, 1.0])
        s_filt[0] = s_hat[0].copy()
        P_filt[0] = P[0].copy()
        
        for k in range(T - 1):
            t_k = master_timestamps[k]
            t_k1 = master_timestamps[k + 1]
            delta_t = t_k1 - t_k
            
            # Prediction step
            s_hat[k + 1] = self.state_transition(s_filt[k], delta_t)
            F_k = self.jacobian_F(delta_t)
            
            # Improved process noise model
            Q_k = np.diag([
                0, 0, 0,  # No noise in position/angle states
                sigma_vx**2 * delta_t,      # Velocity noise
                sigma_vy**2 * delta_t,
                sigma_omega**2 * delta_t    # Angular velocity noise
            ])
            
            P[k + 1] = F_k @ P_filt[k] @ F_k.T + Q_k
            
            # Ensure covariance remains positive definite
            P[k + 1] = (P[k + 1] + P[k + 1].T) / 2
            eigenvals = np.linalg.eigvals(P[k + 1])
            if np.min(eigenvals) < 1e-8:
                P[k + 1] += np.eye(6) * 1e-6
            
            # Update step
            if t_k1 in timestamps:
                idx = timestamps.index(t_k1)
                observed_sensors = data[idx]['observed']
                
                if observed_sensors != 'none':
                    try:
                        H_k1 = self.jacobian_h(s_hat[k + 1], observed_sensors, d)
                        z_pred = self.h(s_hat[k + 1], observed_sensors, d)
                        z_k1 = data[idx]['obs']
                        m_t = len(z_k1)
                        
                        R = sigma_obs**2 * np.eye(m_t)
                        S_k1 = H_k1 @ P[k + 1] @ H_k1.T + R
                        
                        # Ensure S_k1 is well-conditioned
                        S_k1 = (S_k1 + S_k1.T) / 2
                        eigenvals = np.linalg.eigvals(S_k1)
                        if np.min(eigenvals) < 1e-10:
                            S_k1 += np.eye(m_t) * 1e-8
                        
                        # Calculate innovation
                        innovation = z_k1 - z_pred
                        
                        # Compute log likelihood with numerical stability
                        try:
                            L = np.linalg.cholesky(S_k1)
                            log_det = 2 * np.sum(np.log(np.diag(L)))
                            S_inv_innovation = np.linalg.solve(S_k1, innovation)
                            
                            neg_log_likelihood += 0.5 * (m_t * np.log(2 * np.pi) + log_det + 
                                                        innovation @ S_inv_innovation)
                            
                            # Kalman gain and update
                            K_k1 = P[k + 1] @ H_k1.T @ np.linalg.solve(S_k1, np.eye(m_t))
                            s_filt[k + 1] = s_hat[k + 1] + K_k1 @ innovation
                            P_filt[k + 1] = (np.eye(6) - K_k1 @ H_k1) @ P[k + 1]
                            
                            # Ensure covariance remains positive definite
                            P_filt[k + 1] = (P_filt[k + 1] + P_filt[k + 1].T) / 2
                            
                        except np.linalg.LinAlgError:
                            # Fallback: skip this update
                            s_filt[k + 1] = s_hat[k + 1]
                            P_filt[k + 1] = P[k + 1]
                            neg_log_likelihood += 1000  # Penalty for numerical issues
                            
                    except Exception as e:
                        s_filt[k + 1] = s_hat[k + 1]
                        P_filt[k + 1] = P[k + 1]
                        neg_log_likelihood += 1000
                else:
                    s_filt[k + 1] = s_hat[k + 1]
                    P_filt[k + 1] = P[k + 1]
            else:
                s_filt[k + 1] = s_hat[k + 1]
                P_filt[k + 1] = P[k + 1]
        
        return s_filt, P_filt, s_hat, P, neg_log_likelihood
    
    def state_transition(self, s_t, delta_t):
        """State transition function"""
        x, y, theta, vx, vy, omega = s_t
        return np.array([
            x + vx * delta_t,
            y + vy * delta_t,
            theta + omega * delta_t,
            vx, vy, omega
        ])
    
    def jacobian_F(self, delta_t):
        """Jacobian of state transition"""
        return np.array([
            [1, 0, 0, delta_t, 0, 0],
            [0, 1, 0, 0, delta_t, 0],
            [0, 0, 1, 0, 0, delta_t],
            [0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0],
            [0, 0, 0, 0, 0, 1]
        ])
    
    def h(self, s_t, observed_sensors, d):
        """Observation function"""
        x, y, theta = s_t[0], s_t[1], s_t[2]
        if observed_sensors == 'both':
            return np.array([
                x - d * np.sin(theta), y + d * np.cos(theta),
                x + d * np.sin(theta), y - d * np.cos(theta)
            ])
        elif observed_sensors == 'left':
            return np.array([x - d * np.sin(theta), y + d * np.cos(theta)])
        elif observed_sensors == 'right':
            return np.array([x + d * np.sin(theta), y - d * np.cos(theta)])
        else:
            return np.array([])
    
    def jacobian_h(self, s_t, observed_sensors, d):
        """Jacobian of observation function"""
        theta = s_t[2]
        if observed_sensors == 'both':
            return np.array([
                [1, 0, -d * np.cos(theta), 0, 0, 0],
                [0, 1, -d * np.sin(theta), 0, 0, 0],
                [1, 0, d * np.cos(theta), 0, 0, 0],
                [0, 1, d * np.sin(theta), 0, 0, 0]
            ])
        elif observed_sensors == 'left':
            return np.array([
                [1, 0, -d * np.cos(theta), 0, 0, 0],
                [0, 1, -d * np.sin(theta), 0, 0, 0]
            ])
        elif observed_sensors == 'right':
            return np.array([
                [1, 0, d * np.cos(theta), 0, 0, 0],
                [0, 1, d * np.sin(theta), 0, 0, 0]
            ])
        else:
            return np.zeros((0, 6))

# Example usage with your existing code structure
def run_robust_optimization(real_data, param_combinations):
    """
    Main function to run robust optimization with your existing parameter combinations
    """
    ekf = RobustEKF()
    results = []
    
    for i, (sigma_v, sigma_omega, sigma_obs) in enumerate(param_combinations):
        print(f"\n{'='*50}")
        print(f"Parameter Combination {i+1}/{len(param_combinations)}")
        print(f"sigma_v: {sigma_v}, sigma_omega: {sigma_omega}, sigma_obs: {sigma_obs}")
        print(f"{'='*50}")
        
        try:
            # Prepare data
            max_data_points = min(1500, len(real_data))
            data_subset = real_data[:max_data_points]
            timestamps = [entry['timestamp'] for entry in data_subset]
            virtual_timestamps = np.arange(min(timestamps), max(timestamps), ekf.DT_VIRT).tolist()
            
            initial_params = [sigma_v, sigma_v, sigma_omega, sigma_obs, 0.23]
            
            # Run robust cross-validation
            cv_results = ekf.robust_cross_validation(data_subset, timestamps, virtual_timestamps, 
                                                   initial_params, n_splits=3)
            
            if cv_results['best_params'] is None:
                raise ValueError("Cross-validation failed for all folds")
            
            # Validate parameters
            validation_issues = ekf.validate_parameters(cv_results['best_params'])
            
            # Final optimization on full dataset
            best_result, all_optimization_results = ekf.multiple_initialization_optimization(
                data_subset, timestamps, virtual_timestamps, n_initializations=8)
            
            if best_result is None:
                raise ValueError("All optimization attempts failed")
            
            # Create result dictionary
            result_dict = {
                'combination_id': i + 1,
                'initial_sigma_v': sigma_v,
                'initial_sigma_omega': sigma_omega,
                'initial_sigma_obs': sigma_obs,
                'optimized_sigma_vx': best_result['params'][0],
                'optimized_sigma_vy': best_result['params'][1],
                'optimized_sigma_omega': best_result['params'][2],
                'optimized_sigma_obs': best_result['params'][3],
                'optimized_d': best_result['params'][4],
                'negative_log_likelihood': best_result['score'],
                'avg_cv_score': cv_results['avg_val_score'],
                'std_cv_score': cv_results['std_val_score'],
                'n_successful_cv_folds': cv_results['n_successful_folds'],
                'optimization_method': best_result['method'],
                'n_optimization_attempts': len(all_optimization_results),
                'validation_issues': validation_issues,
                'optimization_success': True
            }
            
            results.append(result_dict)
            
            print(f"\nRESULTS:")
            print(f"Best parameters: {best_result['params']}")
            print(f"Negative log-likelihood: {best_result['score']:.4f}")
            print(f"CV score: {cv_results['avg_val_score']:.4f} ± {cv_results['std_cv_score']:.4f}")
            print(f"Optimization method: {best_result['method']}")
            if validation_issues:
                print(f"Parameter validation issues: {validation_issues}")
            
        except Exception as e:
            print(f"ERROR: {str(e)}")
            result_dict = {
                'combination_id': i + 1,
                'initial_sigma_v': sigma_v,
                'initial_sigma_omega': sigma_omega,
                'initial_sigma_obs': sigma_obs,
                'optimized_sigma_vx': np.nan,
                'optimized_sigma_vy': np.nan,
                'optimized_sigma_omega': np.nan,
                'optimized_sigma_obs': np.nan,
                'optimized_d': np.nan,
                'negative_log_likelihood': np.inf,
                'avg_cv_score': np.inf,
                'std_cv_score': np.inf,
                'n_successful_cv_folds': 0,
                'optimization_method': 'failed',
                'n_optimization_attempts': 0,
                'validation_issues': [str(e)],
                'optimization_success': False
            }
            results.append(result_dict)
    
    return results


In [ ]:
# Create the robust EKF instance
ekf = RobustEKF()

# Run with your existing parameter combinations
results = run_robust_optimization(real_data, param_combinations[:5])  # Test with first 5 combinations

# Analyze results
best_result = min([r for r in results if r['optimization_success']], 
                  key=lambda x: x['negative_log_likelihood'])


Parameter Combination 1/5
sigma_v: 0.01, sigma_omega: 0.001, sigma_obs: 0.5
Processing fold 1/3
Optimization attempt 1/5
  Score: 1302.2625, Method: Nelder-Mead
Optimization attempt 2/5
  Score: 1349.4686, Method: L-BFGS-B
Optimization attempt 3/5
  Score: 2845.9748, Method: Nelder-Mead
Optimization attempt 4/5
  Score: 4743.9514, Method: L-BFGS-B
Optimization attempt 5/5
  Score: 2482.4291, Method: Nelder-Mead
Fold 1 - Val score: 753.49, Train score: 1302.26
Processing fold 2/3
Optimization attempt 1/5
  Score: 1266.6444, Method: L-BFGS-B
Optimization attempt 2/5
  Score: 1484.8390, Method: L-BFGS-B
Optimization attempt 3/5
  Score: 2810.0654, Method: Nelder-Mead
Optimization attempt 4/5
  Score: 4089.6048, Method: Nelder-Mead
Optimization attempt 5/5
  Score: 2448.0205, Method: Nelder-Mead
Fold 2 - Val score: 845.30, Train score: 1266.64
Processing fold 3/3
Optimization attempt 1/5
  Score: 1251.7912, Method: L-BFGS-B
Optimization attempt 2/5
  Score: 2183.9739, Method: Nelder-Mead
